In [1]:
import numpy as np
import pandas as pd

In [2]:
matches = pd.read_csv("matches 2019-2023.xls", index_col=0)

In [3]:
matches = matches.fillna(0)
del matches["comp"]
del matches["notes"]
matches["date"] = pd.to_datetime(matches["date"])
matches["target"] = (matches["result"] == "W").astype("int")

In [4]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek

In [5]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group 

cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

In [6]:
matches_rolling

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,hour,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2018-09-02,13:30,Matchweek 4,Sun,Away,W,3.0,2.0,Cardiff City,1.7,...,13,6,1.666667,2.000000,13.666667,6.333333,18.166667,0.333333,0.000000,0.000000
1,2018-09-15,15:00,Matchweek 5,Sat,Away,W,2.0,1.0,Newcastle Utd,1.0,...,15,5,2.666667,2.000000,16.333333,9.000000,17.300000,0.333333,0.000000,0.000000
2,2018-09-23,16:00,Matchweek 6,Sun,Home,W,2.0,0.0,Everton,0.8,...,16,6,2.666667,1.333333,15.333333,7.666667,17.333333,0.666667,0.000000,0.000000
3,2018-09-29,15:00,Matchweek 7,Sat,Home,W,2.0,0.0,Watford,1.4,...,15,5,2.333333,1.000000,12.666667,6.000000,17.133333,0.333333,0.000000,0.000000
4,2018-10-07,12:00,Matchweek 8,Sun,Away,W,5.0,1.0,Fulham,1.1,...,12,6,2.000000,0.333333,10.000000,3.000000,16.666667,0.333333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,2023-04-15,15:00,Matchweek 31,Sat,Home,W,2.0,0.0,Brentford,0.8,...,15,5,1.333333,1.666667,13.333333,4.333333,19.166667,0.333333,0.000000,0.000000
3649,2023-04-22,15:00,Matchweek 32,Sat,Away,L,1.0,2.0,Leicester City,0.9,...,15,5,1.333333,0.333333,9.333333,4.666667,19.933333,0.000000,0.000000,0.000000
3650,2023-04-25,19:30,Matchweek 33,Tue,Home,W,2.0,0.0,Crystal Palace,1.4,...,19,1,1.333333,0.666667,12.000000,5.333333,21.100000,0.666667,0.000000,0.000000
3651,2023-04-29,15:00,Matchweek 34,Sat,Away,L,0.0,6.0,Brighton,0.8,...,15,5,1.666667,0.666667,11.666667,4.666667,18.700000,0.666667,0.333333,0.333333


In [7]:
matches_rolling.to_csv("matches_after_preprocessing.csv")